In [5]:
### 1. 분산을 기준으로 수치 특성 선택하기
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris=datasets.load_iris()
print(iris.data[0:3])
# 특성과 타겟
features=iris.data
target=iris.target

# 분산 기준 설정
thresholder=VarianceThreshold(threshold=0.5)

# 분산 기준값보다 높은 특성을 선택(기준에 미달하는 모든 특성을 삭제함)
features_high_variance=thresholder.fit_transform(features)

# 선택한 특성을 확인
print(features_high_variance[0:3])
# 4개의 변수가 3개로 감소함(두 번째 변수가 삭제됨)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
[[5.1 1.4 0.2]
 [4.9 1.4 0.2]
 [4.7 1.3 0.2]]


In [6]:
# 분산을 확인
thresholder.variances_
# 분산이 낮은 두 번째 변수가 삭제되었음

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

In [7]:
# 이 방법은 분산을 기준으로 변수를 선택하므로 표준화를 한 후에 수행하면 효과가 없음
# 표준화를 하면 평균 0, 표준편차 1로 조정되므로 모든 변수의 분산이 같아지게 되어 효과가 없음
from sklearn.preprocessing import StandardScaler

# 특성 행렬을 표준화
scaler=StandardScaler()
features_std=scaler.fit_transform(features)

# 각 특성의 분산을 계산
selector=VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

In [8]:
### 2. 분산을 기준으로 이진 특성 선택하기
# 범주형 변수의 분산을 조사(이진분류의 경우)
from sklearn.feature_selection import VarianceThreshold

# 예제 특성 행렬
# 특성 0 : 80%가 클래스 0, 20%가 클래스 1
# 특성 1 : 20%가 클래스 0, 80%가 클래스 1
# 특성 2 : 60%가 클래스 0, 40%가 클래스 1
# 0 또는 1이 75% 이상인 변수는 제거하고 그렇지 않은 변수(여기서는 특성 2)를 선택
features=[[0,1,0],[0,1,1],[0,1,0],[0,1,1],[1,0,0]]

# 분산을 기준으로 선택
# p * (1-p) : p는 클래스의 비율
# threshold : 기본값 0 → 모든 특성을 선택함
thresholder=VarianceThreshold(threshold=(0.75*(1-0.75)))
features2=thresholder.fit_transform(features)
print(0.75*(1-0.75))
print(features2)
# 각변수들의 분산이 0.1875보다 큰 변수인 세 번째 변수가 선택됨
# 첫 번째 변수는 클래스 0이 80%, 두 번째 변수는 클래스 1이 80%이므로 제거되고 세 번째 변수는 분산이 높으므로 선택됨
# 기본값은 0 (모든 변수 삭제)

0.1875
[[0]
 [1]
 [0]
 [1]
 [0]]


In [9]:
thresholder.variances_ # 각 변수들의 분산값

array([0.16, 0.16, 0.24])

In [10]:
# 분산을 계산하는 또 다른 방법
import numpy as np
np.var(features, axis=0)

array([0.16, 0.16, 0.24])

In [26]:
### 3. 상관관계가 큰 특성 다루기
# 다중공선성 문제 : 독립변수들끼리 비슷하다면 가지고 있는 정보들이 비슷하므로 중복된 특성을 포함하게 되어
#                  모형의 예측력이 왜곡될 수 있음
import pandas as pd
import numpy as np

# 상관 관계가 큰 두 개의 특성을 가진 특성 행렬
features = np.array([\
[1, 1, 1],
[2, 2, 0],
[3, 3, 1],
[4, 4, 0],
[5, 5, 1],
[6, 6, 0],
[7, 7, 1],
[8, 7, 0],
[9, 7, 1]])

# 특성 행렬을 DataFrame으로 변환
df=pd.DataFrame(features)

# 상관계수 행렬
corr_matrix=df.corr().abs()
print(corr_matrix)

          0         1         2
0  1.000000  0.976103  0.000000
1  0.976103  1.000000  0.034503
2  0.000000  0.034503  1.000000
    0         1         2
0 NaN  0.976103  0.000000
1 NaN       NaN  0.034503
2 NaN       NaN       NaN


In [29]:
# 상관관계 행렬의 상삼각(upper triangle) 행렬을 선택
# 행렬의 대각선을 기준으로 윗부분을 상삼각 행렬, 아랫부분을 하삼각 행렬이라고 함
# triu : 상삼각 행렬(upper triangle)
# k=0 : 대각 원소가 포함(기본 옵션), k=1 : 대각선에서 1만큼 떨어진 삼각 행렬
upper=corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
print(upper)

# 상관 계수가 0.95보다 큰 특성 열의 인덱스
to_drop=[column for column in upper.columns if any(upper[column]>0.95)]
print("\n",to_drop)

# 0과 1 변수의 상관관계가 높음
# 인덱스 1 특성을 삭제
df2=df.drop(df.columns[to_drop], axis=1).head(3)
print("\n",df2)
# 1번이 삭제되고 0, 2번 필드만 선택됨

    0         1         2
0 NaN  0.976103  0.000000
1 NaN       NaN  0.034503
2 NaN       NaN       NaN

 [1]

    0  2
0  1  1
1  2  0
2  3  1


In [31]:
# 넘파이에서 상관 행렬을 구하는 함수
np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

In [33]:
# triu : 상삼각 행렬(upper triangle)
# k=0 : 대각 원소가 포함(기본 옵션), k=1 : 대각선에서 1만큼 떨어진 삼각 행렬
np.triu(np.ones((4,4)), k=2)

array([[0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [34]:
# tril : 하삼각 행렬(lower triangle)
np.tril(np.ones((4,4)), k=0)

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [35]:
### 4. 분류 작업에 관련 없는 특성 삭제하기

# 번주형 변수에서 관련없는 특성 삭제

from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

# 데이터를 로드
iris=datasets.load_iris()
features=iris.data
target=iris.target

# 변수의 자료형을 정수형으로 변환
features=features.astype(int)

# 카이제곱 통계값이 가장 큰 특성 두 개를 선택(범주형 변수에만 사용 가능, 모든 샘플에 음수값이 없어야 함)
chi2_selector=SelectKBest(chi2, k=2) # k : 변수의 수
features_kbest=chi2_selector.fit_transform(features, target)

# 결과를 확인
print("원본 특성 개수 :", features.shape[1])
print("줄어든 특성 개수 :", features_kbest.shape[1])
print(features[:3])
print(features_kbest[:3])
# 0, 1번 변수가 삭제되었음
# 카이제곱 통계량(통계량이 낮은 0, 1번 변수가 삭제됨)
# ※ 통계량 : 표본 그룹의 분산값
print(chi2_selector.scores_)

원본 특성 개수 : 4
줄어든 특성 개수 : 2
[[5 3 1 0]
 [4 3 1 0]
 [4 3 1 0]]
[[1 0]
 [1 0]
 [1 0]]
[ 10.28712871   5.02267003 133.06854839  74.27906977]


In [36]:
# 수치형 변수의 경우 분산분석(ANOVA)의 F값(표본 그룹의 분산)을 계산
# F-값이 가장 높은 특성 두 개를 선택
features=iris.data
fvalue_selector=SelectKBest(f_classif, k=2)
features_kbest=fvalue_selector.fit_transform(features,target)

# 결과를 확인
print("원본 특성 개수 :", features.shape[1])
print("줄어든 특성 개수 :", features_kbest.shape[1])
print(features[:3])
print(features_kbest[:3])
# 0, 1번 변수가 삭제되었음

원본 특성 개수 : 4
줄어든 특성 개수 : 2
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
[[1.4 0.2]
 [1.4 0.2]
 [1.3 0.2]]


In [37]:
from sklearn.feature_selection import SelectPercentile

# 특성 개수 대신 비율로 선택하는 방법
# 가장 큰 F-값의 상위 75% 특성을 선택
fvalue_selector=SelectPercentile(f_classif, percentile=75)
features_kbest=fvalue_selector.fit_transform(features,target)

# 결과를 확인
print("원본 특성 개수 :", features.shape[1])
print("줄어든 특성 개수 :", features_kbest.shape[1])
print(features[:3])
print(features_kbest[:3])
# 1번 변수가 삭제되었음
# 변수별 아노바 F 통계량 ( 가장 낮은 1번 변수가 삭제됨)
print(fvalue_selector.scores_)

원본 특성 개수 : 4
줄어든 특성 개수 : 3
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
[[5.1 1.4 0.2]
 [4.9 1.4 0.2]
 [4.7 1.3 0.2]]
[ 119.26450218   49.16004009 1180.16118225  960.0071468 ]


In [38]:
# 카이제곱 통계 : 두 범주형 벡터의 독립성을 평가하는 지표

# (Oi - Ei)**2 / Ei 의 합계
# Oi : 클래스 i의 샘플 빈도
# Ei : 특성과 target vector 사이에 관계가 없을 때 기대할 수 있는 클래스 i의 샘플 빈도

# iris 데이터셋은 실수형이므로 범주형처럼 처리하기 위하여 정수로 미리 변환하였음
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [39]:
# 150행 4열을 클래스 3개로 나누기 위하여 3차원으로 변환 3품종 × 50샘플 × 4변수
observed=np.sum(features.reshape(3,50,4), axis=1)
observed
# 클래스별 합계값

array([[250.3, 171.4,  73.1,  12.3],
       [296.8, 138.5, 213. ,  66.3],
       [329.4, 148.7, 277.6, 101.3]])

In [40]:
# 기대 빈도 = 전체 합계 / 3(클래스 수)
expected=features.sum(axis=0) / 3
expected

array([292.16666667, 152.86666667, 187.9       ,  59.96666667])

In [41]:
# 카이제곱 공식에 대입하여 얻은 카이제곱 점수
np.sum((observed-expected)**2 / expected, axis=0)

array([ 10.81782088,   3.7107283 , 116.31261309,  67.0483602 ])

In [42]:
# SelectKBest 모형에 저장된 카이제곱 점수
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])